In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext tensorboard
import datetime

!rm -rf ./logs/RNN

#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

In [2]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [3]:
#retrieve and format data - into labels from the dataset
def labels(filename):
    data = np.matrix(pd.read_csv(filename).values)
    data_y = data[:, 0]
    data_y-=1 
    return data_y

In [4]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x) 
training_data_y = labels("../archive/exoTrain.csv")
test_y = labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [ ]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

Epoch 1/3
112/112 [==============================] - 590s 5s/step - loss: 0.3058 - accuracy: 0.9817 - val_loss: 0.2602 - val_accuracy: 0.9876
Epoch 2/3
111/112 [============================>.] - ETA: 4s - loss: 0.2625 - accuracy: 0.9882

In [ ]:
%tensorboard --logdir logs/RNN/fit